In [82]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import re
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from wordcloud import WordCloud

from konlpy import tag
from konlpy.corpus import kolaw
from konlpy.tag import Kkma, Komoran, Okt, Hannanum #Mecab은 윈도우에서 작동 불가능 #형태소 분석기 호출

from nltk.tokenize import sent_tokenize
from collections import Counter #데이터의 개수를 정리할 수 있는 모듈
from konlpy.utils import pprint #유니코드 문자 출력

import warnings
warnings.filterwarnings('ignore')

In [83]:
Lyrics=pd.read_csv('./data/Lyrics_final.csv',encoding='utf-8')
Lyrics.head(2)

,Lyrics,label
0,아침에 일어나 너에게 짧은 인사를 보낸다 아무리 멀리 떨어져 있어도 생각하고 늦은 ...,0
1,맛있었다 고 했던 살구빛 샐러드 맛있어서 먹고 싶어 좋아했던 것들을 나에게 자그마한...,0


노래가사 이진분류 (정답레이블 기준)

In [84]:
def rating_to_label(label):
    if label == 0 or label == 2:
        return 1
    else:
        return 0
    
Lyrics['y'] = Lyrics['label'].apply(lambda x: rating_to_label(x))
Lyrics_pos = Lyrics[Lyrics['y'] == 1]
Lyrics_neg = Lyrics[Lyrics['y'] == 0]
Lyrics_pos

,Lyrics,label,y
0,아침에 일어나 너에게 짧은 인사를 보낸다 아무리 멀리 떨어져 있어도 생각하고 늦은 ...,0,1
1,맛있었다 고 했던 살구빛 샐러드 맛있어서 먹고 싶어 좋아했던 것들을 나에게 자그마한...,0,1
2,마주치는 눈빛에 아기 아기 왠지 어색해 대체 고개를 숙여봐도 발끝에 맘이 설레 두근...,0,1
3,어깨동무까진 괜찮아 치마 손잡는 건 위험해 잘 자란 메세 진 괜찮아그래도 자냐는 메...,0,1
4,햇살이 가득 그댈 향한 가벼운 발걸음 콧노래 부르며 쇼윈도에 비친 모습을 한번 점검...,0,1
...,...,...,...
591,푸른 새벽 잠들지 못하는 보며 꿈의 무게를 작은 한숨이 들릴 때면 안아주고 싶었어 ...,2,1
592,세상이란 장애물이 앞길을 가로막을 언제나 곁엔 불안 따윈 도 없을 테니 이렇게 언제...,2,1
593,온 마음이 우리들을 위로해 온 우주가 우리들을 감싸네 쓸쓸하다고 생각했지 살아가는 ...,2,1
594,돌아서지 않을게 곁 옆에 머물러 함께할 테니 기쁜 날은 뜨거운 눈물과 그대 힘든 날...,2,1


## KNU
긍정,부정 감성 함수 구현

In [85]:
pos_word=pd.read_csv('./pos_pol_word.txt',index_col=0,header=None)
pos_word=pd.DataFrame(pos_word).reset_index()
pos_word.columns=['pos_word']

neg_word=pd.read_csv('./neg_pol_word.txt',index_col=0,header=None)
neg_word=pd.DataFrame(neg_word).reset_index()
neg_word.columns=['neg_word']

In [86]:
def LyricsWordCount(arrList, emotion):

    list = []
    result = []
    i=1
    knu_num=[]

    # 감성 점수 구하기
    for Lyrics in arrList:
        neg_count = 0       # 부정 점수
        pos_count = 0       # 긍정 점수
        num = 0             # 감성 점수 
        
        # 긍정 점수 구하기
        for pos_text in pos_word['pos_word']:
            pos_find = Lyrics.find(pos_text)
            if(pos_find > -1):
                pos_count += 1
            else: pos_count += 0


        # 부정 점수 구하기
        for neg_text in neg_word['neg_word']:
            neg_find = Lyrics.find(neg_text)
            if(neg_find > -1):
                neg_count += 1
            else: neg_count += 0

        # 감성 점수 공식
        if emotion == '긍정':
          if pos_count == 0 or neg_count == 0:
            num = 0
          else:
            num = pos_count/(pos_count + neg_count)
            
        elif emotion == '부정':
          if pos_count == 0 or neg_count == 0:
            num = 0
          else:
            num = neg_count/(pos_count + neg_count)
        knu_num.append(num)

        # 긍, 부정 결과
        if emotion == '긍정':
          if num >= 0.5:
            result = '긍정'
          elif num < 0.5:
            result = '부정'
        
        if emotion == '부정':
          if num >= 0.5:
            result = '부정'
          elif num < 0.5:
            result = '긍정'

        srt = f"{i}번 노래 결과 -> 긍정 : {pos_count}, 부정 : {neg_count} , 감성 점수 : {round(num,3)}  {result}"
        i = i + 1
        list.append(srt)
    return list,knu_num

In [87]:
pos_result,knu_pos_num = LyricsWordCount(Lyrics_pos['Lyrics'], "긍정")
neg_result,knu_neg_num = LyricsWordCount(Lyrics_neg['Lyrics'], "부정")

In [88]:
Lyrics_pos['knu']=knu_pos_num
Lyrics_pos.head()

,Lyrics,label,y,knu
0,아침에 일어나 너에게 짧은 인사를 보낸다 아무리 멀리 떨어져 있어도 생각하고 늦은 ...,0,1,0.625000
1,맛있었다 고 했던 살구빛 샐러드 맛있어서 먹고 싶어 좋아했던 것들을 나에게 자그마한...,0,1,0.000000
2,마주치는 눈빛에 아기 아기 왠지 어색해 대체 고개를 숙여봐도 발끝에 맘이 설레 두근...,0,1,0.571429
3,어깨동무까진 괜찮아 치마 손잡는 건 위험해 잘 자란 메세 진 괜찮아그래도 자냐는 메...,0,1,0.700000
4,햇살이 가득 그댈 향한 가벼운 발걸음 콧노래 부르며 쇼윈도에 비친 모습을 한번 점검...,0,1,0.857143


In [89]:
Lyrics_neg['knu']=knu_neg_num
Lyrics_neg.head()

,Lyrics,label,y,knu
196,사랑했던 나라 가끔은 없이도 영화를 보고 가끔은 혼자서도 술을 마셨어 너와 다툴 때...,1,0,0.533333
197,아무 말 하지 않아도 그대 눈빛을 보면 전해지는 시린 마음 깊게 잠긴 목소리 옅은 ...,1,0,0.500000
198,어떨 같아 남이 된다 면 서로를 놓아주게 된다면 다를 있니 우리를 늘 곳만 보고 있...,1,0,0.666667
199,우리에게 남은 질문은 없었어할 말도 없고 어색한 공기에 필요 없는 말도 했어 평소 ...,1,0,0.470588
200,너에게 사람이 생겼다는 소식을 듣게 되었어난 하루 종일 복잡한 마음이라서 아무것도 ...,1,0,0.363636


## KoSac

In [90]:
import pandas as pd
import numpy as np

import re
from konlpy.tag import Okt
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from konlpy.tag import Kkma, Komoran, Okt, Hannanum  #Mecab은 윈도우에서 작동 불가능

from konlpy import tag
from konlpy.corpus import kolaw
# from konlpy.utils import csvwrite, pprint

from nltk.tokenize import sent_tokenize

In [91]:
kosac=pd.read_csv('./polarity.csv',encoding='utf-8')
kosac

,ngram,freq,COMP,NEG,NEUT,None,POS,max.value,max.prop
0,가*/JKS,1,0.0,0.0,0.0,0.0,1.0,POS,1.0
1,가*/JKS;있/VV,1,0.0,0.0,0.0,0.0,1.0,POS,1.0
2,가*/JKS;있/VV;었/EP,1,0.0,0.0,0.0,0.0,1.0,POS,1.0
3,가*/VV,3,0.0,0.0,0.0,0.0,1.0,POS,1.0
4,가*/VV;ㄴ다*/EF,1,0.0,0.0,0.0,0.0,1.0,POS,1.0
...,...,...,...,...,...,...,...,...,...
16357,힘들/VA;ㄹ/ETM;것/NNB,1,0.0,1.0,0.0,0.0,0.0,NEG,1.0
16358,힘들/VA;ㄹ/ETM;때/NNG,1,0.0,1.0,0.0,0.0,0.0,NEG,1.0
16359,힘차/VA,1,0.0,1.0,0.0,0.0,0.0,NEG,1.0
16360,힘차/VA;ㄴ/ETM,1,0.0,1.0,0.0,0.0,0.0,NEG,1.0


In [92]:
kkma=Kkma()

data_pos = Lyrics_pos['Lyrics'].to_string()
data_neg = Lyrics_neg['Lyrics'].to_string()

kkma_pos = kkma.pos(data_pos)
kkma_neg = kkma.pos(data_neg)

In [93]:
kkma_pos_list = []
kkma_neg_list = []

for i in range(len(kkma_pos)):
  pos_text = kkma_pos[i][0]+'/'+kkma_pos[i][1]
  kkma_pos_list.append(pos_text)
  i += 1

for i in range(len(kkma_neg)):
  neg_text = kkma_neg[i][0]+'/'+kkma_neg[i][1]
  kkma_neg_list.append(neg_text)
  i += 1

In [94]:
# 형태소분석이 된 긍정가사를 가사별로 리스트화하여 dataframe으로 정리
text_pos = ''.join(str(i) for i in kkma_pos_list)
text_neg = ' '.join(str(i) for i in kkma_neg_list)

text_pos_list = []
numList_pos = []

pattern = "[0-9]"

numList = re.findall(r'\d+', text_pos)

for i in numList:
  if(text_pos.find(f'{i}') > -1):
    num_pos = text_pos.find(f'{i}')
    numList_pos.append(num_pos)

for i in range(len(numList_pos)):
  if(i == len(numList_pos)-1):
    text_2 = text_pos[numList_pos[i]:len(text_pos)]
  else:
    text_2 = text_pos[numList_pos[i]:numList_pos[i+1]]
  text_pos_list.append(text_2)

pos = pd.DataFrame(text_pos_list)

# 형태소분석이 된 부정가사를 가사별로 리스트화하여 dataframe으로 정리
text_neg_list = []
numList_neg = []

pattern = "[0-9]"

numList = re.findall(r'\d+', text_neg)

for i in numList:
  if(text_neg.find(f'{i}') > -1):
    num_neg = text_neg.find(f'{i}')
    numList_neg.append(num_neg)

for i in range(len(numList_neg)):
  if(i == len(numList_neg)-1):
    text_1 = text_neg[numList_neg[i]:len(text_neg)]
  else:
    text_1 = text_neg[numList_neg[i]:numList_neg[i+1]]
  text_neg_list.append(text_1)

neg = pd.DataFrame(text_neg_list)

neg.columns = ['word']
pos.columns = ['word']
pos

,word
0,0/NR아침/NNG에/JKM일어/NNG나/JX너/NP에게/JKM짧/VA은/ETD인사...
1,1/NR맛있/VA었/EPT다/ECS이/VCP고/ECE하/VV었/EPT더/EPTㄴ/E...
2,2/NR마주치/VV는/ETD눈빛/NNG에/JKM아기/NNG아기/NNG왠지/MAG어색...
3,3/NR어깨동무/NNG까지/JX는/JX괜찮/VA아/ECD치마/NNG손잡/VV는/ET...
4,4/NR햇살/NNG이/JKS가득/MAG그/MDT대/VVㄹ/ETD향/NNG하/XSVㄴ...
...,...
391,591/NR푸른/NNP새벽/NNG잠들/VVㄹ지/ECD못하/VX는/ETD보/VV며/E...
392,592/NR세상/NNG이/VCP란/ETD장애물/NNG이/JKS앞길/NNG을/JKO가...
393,593/NR오/VVㄴ/ETD마음/NNG이/JKS우리/NP들/XSN을/JKO위로/NN...
394,594/NR돌아서/VV지/ECD않/VXV을/ETD것/NNB이/JKS곁/NNG옆/NN...


In [95]:
neg

,word
0,196/NR 사랑/NNG 하/XSV 었/EPT 더/EPT ㄴ/ETD 나라/NNG 가...
1,197/NR 아무/MDT 말/NNG 하/VV 지/ECD 않/VXV 아도/ECD 그대...
2,198/NR 어떠/VV ㄹ/ETD 같/VA 아/ECD 남/NNG 이/JKC 되/VV...
3,199/NR 우리/NP 에게/JKM 남/VV 은/ETD 질문/NNG 은/JX 없었/...
4,200/NR 너/NP 에게/JKM 사람/NNG 이/JKS 생기/VV 었/EPT 다는...
...,...
345,741/NR 반/NNG 쯤/NNG 미치/VV 어/ECS 갈/VV 는/ETD 망가지/...
346,742/NR 무/NNG 궁/NNG 화/XSN 꽃/NNG 이/JKS 피/VV 었/EP...
347,743/NR 아/IC 예/IC 아예/MAG 아예/MAG 이렇게/MAG 감/NNG 을...
348,744/NR 나쁘/VA ㄴ/ETD 계집애/NNG 나/VV 아/ECS 나/NP 는/J...


In [96]:
# kosac을 POS 와 NEG로 분류
kosac_pos = kosac[kosac['max.value'] == 'POS']
kosac_neg = kosac[kosac['max.value'] == 'NEG']

In [97]:
def LyricsWordCount(arrList, emotion):
    list = []
    kosac_num=[]
    i=1

    # 감성 점수 구하기
    for Lyrics in arrList:
        neg_count = 0       # 부정 점수
        pos_count = 0       # 긍정 점수
        num = 0             # 감성 점수
        
        # 긍정 점수 구하기
        for pos_text in kosac_pos['ngram']:
            pos_find = Lyrics.find(pos_text)
            if(pos_find > -1):
                pos_count += 1
            else: pos_count += 0


        # 부정 점수 구하기
        for neg_text in kosac_neg['ngram']:
            neg_find = Lyrics.find(neg_text)
            if(neg_find > -1):
                neg_count += 1
            else: neg_count += 0

        # 감성 점수 공식
        if emotion == '긍정':
            num = pos_count - neg_count
        elif emotion == '부정':
            num = neg_count - pos_count

        # 감성 점수 공식
        if emotion == '긍정':
          if pos_count == 0 or neg_count == 0:
            num = 0
          else:
            num = pos_count/(pos_count + neg_count)
            
        elif emotion == '부정':
          if pos_count == 0 or neg_count == 0:
            num = 0
          else:
            num = neg_count/(pos_count + neg_count)

        kosac_num.append(num)

        # 긍, 부정 결과
        if emotion == '긍정':
          if num >= 0.5:
            result = '긍정'
          elif num < 0.5:
            result = '부정'
        
        if emotion == '부정':
          if num >= 0.5:
            result = '부정'
          elif num < 0.5:
            result = '긍정'


        srt = f"{i}번 노래 결과 : 긍정 : {pos_count}, 부정 : {neg_count} , 감성 점수 : {round(num,3)}  {result}"
        i = i + 1

        list.append(srt)
    return list,kosac_num


In [98]:
pos_result,kosac_pos_num = LyricsWordCount(pos['word'], "긍정")
neg_result,kosac_neg_num = LyricsWordCount(neg['word'], "부정")

In [99]:
Lyrics_pos['kosac']=kosac_pos_num
Lyrics_pos.head()

,Lyrics,label,y,knu,kosac
0,아침에 일어나 너에게 짧은 인사를 보낸다 아무리 멀리 떨어져 있어도 생각하고 늦은 ...,0,1,0.625000,0.500000
1,맛있었다 고 했던 살구빛 샐러드 맛있어서 먹고 싶어 좋아했던 것들을 나에게 자그마한...,0,1,0.000000,0.437500
2,마주치는 눈빛에 아기 아기 왠지 어색해 대체 고개를 숙여봐도 발끝에 맘이 설레 두근...,0,1,0.571429,0.571429
3,어깨동무까진 괜찮아 치마 손잡는 건 위험해 잘 자란 메세 진 괜찮아그래도 자냐는 메...,0,1,0.700000,0.416667
4,햇살이 가득 그댈 향한 가벼운 발걸음 콧노래 부르며 쇼윈도에 비친 모습을 한번 점검...,0,1,0.857143,0.545455


In [100]:
Lyrics_neg['kosac']=kosac_neg_num
Lyrics_neg.head()

,Lyrics,label,y,knu,kosac
196,사랑했던 나라 가끔은 없이도 영화를 보고 가끔은 혼자서도 술을 마셨어 너와 다툴 때...,1,0,0.533333,0.529412
197,아무 말 하지 않아도 그대 눈빛을 보면 전해지는 시린 마음 깊게 잠긴 목소리 옅은 ...,1,0,0.500000,0.500000
198,어떨 같아 남이 된다 면 서로를 놓아주게 된다면 다를 있니 우리를 늘 곳만 보고 있...,1,0,0.666667,0.526316
199,우리에게 남은 질문은 없었어할 말도 없고 어색한 공기에 필요 없는 말도 했어 평소 ...,1,0,0.470588,0.500000
200,너에게 사람이 생겼다는 소식을 듣게 되었어난 하루 종일 복잡한 마음이라서 아무것도 ...,1,0,0.363636,0.533333


In [101]:
Lyrics=pd.concat([Lyrics_pos,Lyrics_neg])
Lyrics

,Lyrics,label,y,knu,kosac
0,아침에 일어나 너에게 짧은 인사를 보낸다 아무리 멀리 떨어져 있어도 생각하고 늦은 ...,0,1,0.625000,0.500000
1,맛있었다 고 했던 살구빛 샐러드 맛있어서 먹고 싶어 좋아했던 것들을 나에게 자그마한...,0,1,0.000000,0.437500
2,마주치는 눈빛에 아기 아기 왠지 어색해 대체 고개를 숙여봐도 발끝에 맘이 설레 두근...,0,1,0.571429,0.571429
3,어깨동무까진 괜찮아 치마 손잡는 건 위험해 잘 자란 메세 진 괜찮아그래도 자냐는 메...,0,1,0.700000,0.416667
4,햇살이 가득 그댈 향한 가벼운 발걸음 콧노래 부르며 쇼윈도에 비친 모습을 한번 점검...,0,1,0.857143,0.545455
...,...,...,...,...,...
741,반쯤 미쳐가는 망가져 삶넌 내게서 지워가고 부숴버려라 추락한 눈앞에서 멀어지는 속도...,3,0,0.600000,0.476190
742,무궁화 꽃이 피었습니다 빼고 멈춰지담 꽃이 피었습니다 현아 빼고 비켜 도망가 너나 ...,3,0,0.571429,0.384615
743,아예 아예 아예 이렇게 감을 못 잡는 말을 하는지 모르겠어 뭐야 싱긋싱긋 웃으며 치...,3,0,0.750000,0.750000
744,나쁜 계집애 나난 나쁜 계집애 나쁜 계집애 모든 여자가 고난 나난 모든 여자가 곳 ...,3,0,0.555556,0.666667


## knu, kosac 평균

In [102]:
Lyrics['result']=(Lyrics['knu']+Lyrics['kosac'])/2
Lyrics.head(2)

,Lyrics,label,y,knu,kosac,result
0,아침에 일어나 너에게 짧은 인사를 보낸다 아무리 멀리 떨어져 있어도 생각하고 늦은 ...,0,1,0.625,0.5000,0.56250
1,맛있었다 고 했던 살구빛 샐러드 맛있어서 먹고 싶어 좋아했던 것들을 나에게 자그마한...,0,1,0.000,0.4375,0.21875


In [103]:
def pos_neg(x):
    if x['result']>=0.5:
        return '1'
    else:
        return '0'

Lyrics['pos']=Lyrics.apply(pos_neg,axis=1)
Lyrics.head()

,Lyrics,label,y,knu,kosac,result,pos
0,아침에 일어나 너에게 짧은 인사를 보낸다 아무리 멀리 떨어져 있어도 생각하고 늦은 ...,0,1,0.625000,0.500000,0.562500,1
1,맛있었다 고 했던 살구빛 샐러드 맛있어서 먹고 싶어 좋아했던 것들을 나에게 자그마한...,0,1,0.000000,0.437500,0.218750,0
2,마주치는 눈빛에 아기 아기 왠지 어색해 대체 고개를 숙여봐도 발끝에 맘이 설레 두근...,0,1,0.571429,0.571429,0.571429,1
3,어깨동무까진 괜찮아 치마 손잡는 건 위험해 잘 자란 메세 진 괜찮아그래도 자냐는 메...,0,1,0.700000,0.416667,0.558333,1
4,햇살이 가득 그댈 향한 가벼운 발걸음 콧노래 부르며 쇼윈도에 비친 모습을 한번 점검...,0,1,0.857143,0.545455,0.701299,1


In [115]:
cnt=0
for i in range(len(Lyrics)):
    if Lyrics['y'][i]==int(Lyrics['pos'][i]):
        cnt+=1
print('전체 정확도 : {:.2f}'.format(cnt/len(Lyrics)))

pos_cnt=0
cnt=0
for i in range(len(Lyrics)):
    if Lyrics['y'][i]==1:
        cnt+=1
        if Lyrics['y'][i]==int(Lyrics['pos'][i]):
            pos_cnt+=1
print('긍정 정확도: {:.2f}'.format(pos_cnt/cnt))

neg_cnt=0
cnt=0
for i in range(len(Lyrics)):
    if Lyrics['y'][i]==0:
        cnt+=1
        if Lyrics['y'][i]==int(Lyrics['pos'][i]):
            neg_cnt+=1
print('부정 정확도: {:.2f}'.format(neg_cnt/cnt))

전체 정확도 : 0.46
긍정 정확도: 0.57
부정 정확도: 0.35


In [116]:
Lyrics.to_csv('./data/Lyrics_knu_kosac.csv',encoding='utf-8-sig',index=False)